In [1]:
import re
import qiskit
import numpy as np
from typing import List
from qiskit import *
import time


In [2]:
from circuit_generator import generate_opt_circuit, construct_qcc_circuit, simulate_expectationval, update_observables \
             ,fc_tree_circuit, fc_tree_lookahead_circuit, fc_tree_commute_circuit, fc_tree_commute_lookahead_circuit
from utils import compare_lists
from fast_generator import fc_tree_lookahead_circuit as fc_tree_lookahead_fast
from fast_generator import fc_tree_commute_circuit as fc_tree_commute_fast
from fast_generator import fc_tree_commute_lookahead_fast, fc_tree_sort_lookahead_fast, fc_tree_sort_lookahead_estimate
import numpy as np
from rustiq import pauli_network_synthesis, Metric
from rustiq.utils import entangling_count, entangling_depth

In [3]:
from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers

In [11]:
electrons_list = [2, 2, 4, 6, 8, 10, 12]
orbitals_list = [4, 6, 8, 12, 16, 20, 24]

In [ ]:
import json
import time
from qiskit import transpile

results = []

for i, (x, y) in enumerate(zip(electrons_list, orbitals_list)):
    test_paulis = generate_UCCSD_entanglers(x, y)
    test_params = [0.01 * i for i in range(len(test_paulis))]
    
    # Measure time for our method
    start_time = time.time()
    opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=10)
    end_time = time.time()
    our_time = end_time - start_time

    # Measure time for our method
    start_time = time.time()
    opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_sort_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=10)
    end_time = time.time()
    our_time2 = end_time - start_time
    
    # Measure time for Qiskit method
    start_time = time.time()
    origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
    origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    end_time = time.time()
    qiskit_time = end_time - start_time
    
    # # Measure time for RustiQ method
    # start_time = time.time()
    # circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True)
    # end_time = time.time()
    # rustiq_time = end_time - start_time
    
    # Collect results
    result = {
        "electrons": x,
        "orbitals": y,
        "num_paulis": len(test_paulis),
        "times": {
            "our_time": our_time,
            "qiskit_time": qiskit_time,
            "out_time_nosort": our_time2
            # "rustiq_time": rustiq_time
        },
        "gate_counts": {
            "our_method": opt_qc_f.count_ops().get('cx', 0),
            "our_method_nosort": opt_qc_f2.count_ops().get('cx', 0),
            "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
            # "rustiq_method": entangling_count(circuit)
        },
        "test_paulis_file": f"test_paulis_{i}.json"
    }
    print(result)
    results.append(result)
    
    # Save test_paulis to a separate JSON file
    with open(f'benchmarks/test_paulis_{i}.json', 'w') as paulis_file:
        json.dump(test_paulis, paulis_file, indent=4)

# Write results to a JSON file
with open('benchmarks/UCCSD_results.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)

print("Results saved to results.json and test_paulis files")



{'electrons': 2, 'orbitals': 4, 'num_paulis': 24, 'times': {'our_time': 0.336590051651001, 'qiskit_time': 0.6914329528808594, 'out_time_nosort': 0.15126705169677734}, 'gate_counts': {'our_method': 28, 'our_method_nosort': 28, 'qiskit_method': 41}, 'test_paulis_file': 'test_paulis_0.json'}
{'electrons': 2, 'orbitals': 6, 'num_paulis': 80, 'times': {'our_time': 0.9254190921783447, 'qiskit_time': 0.7186391353607178, 'out_time_nosort': 0.844977855682373}, 'gate_counts': {'our_method': 113, 'our_method_nosort': 116, 'qiskit_method': 181}, 'test_paulis_file': 'test_paulis_1.json'}
{'electrons': 4, 'orbitals': 8, 'num_paulis': 320, 'times': {'our_time': 4.285698175430298, 'qiskit_time': 2.3638668060302734, 'out_time_nosort': 3.1462509632110596}, 'gate_counts': {'our_method': 498, 'our_method_nosort': 537, 'qiskit_method': 1003}, 'test_paulis_file': 'test_paulis_2.json'}
{'electrons': 6, 'orbitals': 12, 'num_paulis': 1656, 'times': {'our_time': 27.1879780292511, 'qiskit_time': 18.1285021305084

KeyboardInterrupt: 

In [ ]:
import json
import time
from qiskit import transpile

results = []

for i, (x, y) in enumerate(zip(electrons_list, orbitals_list)):
    test_paulis = generate_UCCSD_entanglers(x, y)
    test_params = [0.01 * i for i in range(len(test_paulis))]
    
    # Measure time for our method
    start_time = time.time()
    opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=5)
    end_time = time.time()
    our_time = end_time - start_time
    
    # Measure time for Qiskit method
    start_time = time.time()
    origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
    origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    end_time = time.time()
    qiskit_time = end_time - start_time
    
    # # Measure time for RustiQ method
    # start_time = time.time()
    # circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True)
    # end_time = time.time()
    # rustiq_time = end_time - start_time
    
    # Collect results
    result = {
        "electrons": x,
        "orbitals": y,
        "num_paulis": len(test_paulis),
        "times": {
            "our_time": our_time,
            "qiskit_time": qiskit_time,
            # "rustiq_time": rustiq_time
        },
        "gate_counts": {
            "our_method": opt_qc_f.count_ops().get('cx', 0),
            "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
            # "rustiq_method": entangling_count(circuit)
        },
        "test_paulis_file": f"test_paulis_{i}.json"
    }
    print(result)
    results.append(result)
    
    # Save test_paulis to a separate JSON file
    with open(f'benchmarks/test_paulis_{i}.json', 'w') as paulis_file:
        json.dump(test_paulis, paulis_file, indent=4)

# Write results to a JSON file
with open('benchmarks/UCCSD_results.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)

print("Results saved to results.json and test_paulis files")



{'electrons': 2, 'orbitals': 4, 'num_paulis': 24, 'times': {'our_time': 0.15595698356628418, 'qiskit_time': 0.11919593811035156}, 'gate_counts': {'our_method': 29, 'qiskit_method': 41}, 'test_paulis_file': 'test_paulis_0.json'}
{'electrons': 2, 'orbitals': 6, 'num_paulis': 80, 'times': {'our_time': 0.5268552303314209, 'qiskit_time': 0.6523592472076416}, 'gate_counts': {'our_method': 115, 'qiskit_method': 181}, 'test_paulis_file': 'test_paulis_1.json'}
{'electrons': 4, 'orbitals': 8, 'num_paulis': 320, 'times': {'our_time': 3.1834049224853516, 'qiskit_time': 2.1649789810180664}, 'gate_counts': {'our_method': 534, 'qiskit_method': 1003}, 'test_paulis_file': 'test_paulis_2.json'}
{'electrons': 6, 'orbitals': 12, 'num_paulis': 1656, 'times': {'our_time': 15.822204113006592, 'qiskit_time': 16.826032161712646}, 'gate_counts': {'our_method': 3151, 'qiskit_method': 5723}, 'test_paulis_file': 'test_paulis_3.json'}
{'electrons': 8, 'orbitals': 16, 'num_paulis': 5376, 'times': {'our_time': 64.809

In [ ]:
import json
import time
from qiskit import transpile

results = []

for i, (x, y) in enumerate(zip(electrons_list, orbitals_list)):
    test_paulis = generate_UCCSD_entanglers(x, y)
    test_params = [0.01 * i for i in range(len(test_paulis))]
    
    # Measure time for our method
    start_time = time.time()
    opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=10)
    end_time = time.time()
    our_time = end_time - start_time

    # Measure time for our method
    start_time = time.time()
    opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_sort_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=10)
    end_time = time.time()
    our_time2 = end_time - start_time
    
    # Measure time for Qiskit method
    start_time = time.time()
    origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
    origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    end_time = time.time()
    qiskit_time = end_time - start_time
    
    # # Measure time for RustiQ method
    # start_time = time.time()
    # circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True)
    # end_time = time.time()
    # rustiq_time = end_time - start_time
    
    # Collect results
    result = {
        "electrons": x,
        "orbitals": y,
        "num_paulis": len(test_paulis),
        "times": {
            "our_time": our_time,
            "qiskit_time": qiskit_time,
            "out_time_nosort": our_time2
            # "rustiq_time": rustiq_time
        },
        "gate_counts": {
            "our_method": opt_qc_f.count_ops().get('cx', 0),
            "our_method_nosort": opt_qc_f2.count_ops().get('cx', 0),
            "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
            # "rustiq_method": entangling_count(circuit)
        },
        "test_paulis_file": f"test_paulis_{i}.json"
    }
    print(result)
    results.append(result)
    
    # Save test_paulis to a separate JSON file
    with open(f'benchmarks/test_paulis_{i}.json', 'w') as paulis_file:
        json.dump(test_paulis, paulis_file, indent=4)

# Write results to a JSON file
with open('benchmarks/UCCSD_results.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)

print("Results saved to results.json and test_paulis files")



curr_pauli YZXI lookahead_entanglers ['XZXI', 'YZYI', 'XZYI', 'IYZX', 'IXZX', 'IYZY', 'IXZY', 'YXYX', 'XXYX', 'YYYX']
curr_pauli YIII lookahead_entanglers ['YZYI', 'XZYI', 'IYZX', 'IXZX', 'IYZY', 'IXZY', 'YXYX', 'XXYX', 'YYYX', 'XYYX']
curr_pauli ZIXI lookahead_entanglers ['XZYI', 'IYZX', 'IXZX', 'IYZY', 'IXZY', 'YXYX', 'XXYX', 'YYYX', 'XYYX', 'YXYY']
curr_pauli XIZI lookahead_entanglers ['IYZX', 'IXZX', 'IYZY', 'IXZY', 'YXYX', 'XXYX', 'YYYX', 'XYYX', 'YXYY', 'XXYY']
curr_pauli IYIX lookahead_entanglers ['IXZX', 'IYZY', 'IXZY', 'YXYX', 'XXYX', 'YYYX', 'XYYX', 'YXYY', 'XXYY', 'YYYY']
curr_pauli IYII lookahead_entanglers ['IYZY', 'IXZY', 'YXYX', 'XXYX', 'YYYX', 'XYYX', 'YXYY', 'XXYY', 'YYYY', 'XYYY']
curr_pauli IZIX lookahead_entanglers ['IXZY', 'YXYX', 'XXYX', 'YYYX', 'XYYX', 'YXYY', 'XXYY', 'YYYY', 'XYYY', 'YXXX']
curr_pauli IXIZ lookahead_entanglers ['YXYX', 'XXYX', 'YYYX', 'XYYX', 'YXYY', 'XXYY', 'YYYY', 'XYYY', 'YXXX', 'XXXX']
curr_pauli ZZZI lookahead_entanglers ['XXYX', 'YYYX', 'X

KeyboardInterrupt: 

In [12]:
import json
import time
from qiskit import transpile

results = []

for i, (x, y) in enumerate(zip(electrons_list, orbitals_list)):
    test_paulis = generate_UCCSD_entanglers(x, y)
    test_params = [0.01 * i for i in range(len(test_paulis))]
    
    # Measure time for our method
    start_time = time.time()
    opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=5)
    end_time = time.time()
    our_time = end_time - start_time

    # Measure time for our method
    start_time = time.time()
    opt_qc_f3, append_clifford_f3, sorted_entanglers_f3 = fc_tree_sort_lookahead_estimate(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=5)
    end_time = time.time()
    our_time3 = end_time - start_time

    start_time = time.time()
    opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=5)
    opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    end_time = time.time()
    combined_time = end_time - start_time
    
    # Measure time for Qiskit method
    start_time = time.time()
    origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
    origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    end_time = time.time()
    qiskit_time = end_time - start_time
    
    # Measure time for RustiQ method
    start_time = time.time()
    circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True)
    end_time = time.time()
    rustiq_time = end_time - start_time
    
    # Collect results
    result = {
        "electrons": x,
        "orbitals": y,
        "num_paulis": len(test_paulis),
        "times": {
            "our_time": our_time,
            "combined_time": combined_time,
            "sort_estimate_time": our_time3,
            "qiskit_time": qiskit_time,
            "rustiq_time": rustiq_time
        },
        "gate_counts": {
            "our_method": opt_qc_f.count_ops().get('cx', 0),
            "combined_method": opt_qiskit.count_ops().get('cx', 0),
            "sort_estimate_time": opt_qc_f3.count_ops().get('cx', 0),
            "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
            "rustiq_method": entangling_count(circuit)
        },
        "test_paulis_file": f"test_paulis_{i}.json"
    }
    print(result)
    results.append(result)
    
#     # Save test_paulis to a separate JSON file
#     with open(f'benchmarks/test_paulis_{i}.json', 'w') as paulis_file:
#         json.dump(test_paulis, paulis_file, indent=4)

# # Write results to a JSON file
# with open('benchmarks/UCCSD_results.json', 'w') as json_file:
#     json.dump(results, json_file, indent=4)

print("Results saved to results.json and test_paulis files")



{'electrons': 2, 'orbitals': 4, 'num_paulis': 24, 'times': {'our_time': 0.1606919765472412, 'combined_time': 0.20881199836730957, 'sort_estimate_time': 0.11857891082763672, 'qiskit_time': 0.16361594200134277, 'rustiq_time': 0.00034689903259277344}, 'gate_counts': {'our_method': 30, 'combined_method': 27, 'sort_estimate_time': 30, 'qiskit_method': 41, 'rustiq_method': 26}, 'test_paulis_file': 'test_paulis_0.json'}
{'electrons': 2, 'orbitals': 6, 'num_paulis': 80, 'times': {'our_time': 0.6953558921813965, 'combined_time': 0.8403961658477783, 'sort_estimate_time': 0.5230789184570312, 'qiskit_time': 0.5210869312286377, 'rustiq_time': 0.003362894058227539}, 'gate_counts': {'our_method': 115, 'combined_method': 99, 'sort_estimate_time': 120, 'qiskit_method': 181, 'rustiq_method': 141}, 'test_paulis_file': 'test_paulis_1.json'}
{'electrons': 4, 'orbitals': 8, 'num_paulis': 320, 'times': {'our_time': 2.8059239387512207, 'combined_time': 3.27911114692688, 'sort_estimate_time': 2.334300994873047

KeyboardInterrupt: 

In [ ]:
import json
import time
from qiskit import transpile

results = []

for i, (x, y) in enumerate(zip(electrons_list, orbitals_list)):
    test_paulis = generate_UCCSD_entanglers(x, y)
    test_params = [0.01 * i for i in range(len(test_paulis))]
    
    # Measure time for our method
    start_time = time.time()
    opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=5)
    end_time = time.time()
    our_time = end_time - start_time

    start_time = time.time()
    opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=5)
    opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    end_time = time.time()
    combined_time = end_time - start_time
    
    # Measure time for Qiskit method
    start_time = time.time()
    origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
    origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    end_time = time.time()
    qiskit_time = end_time - start_time
    
    # Measure time for RustiQ method
    start_time = time.time()
    circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True)
    end_time = time.time()
    rustiq_time = end_time - start_time
    
    # Collect results
    result = {
        "electrons": x,
        "orbitals": y,
        "num_paulis": len(test_paulis),
        "times": {
            "our_time": our_time,
            "combined_time": combined_time,
            "qiskit_time": qiskit_time,
            "rustiq_time": rustiq_time
        },
        "gate_counts": {
            "our_method": opt_qc_f.count_ops().get('cx', 0),
            "combined_method": opt_qiskit.count_ops().get('cx', 0),
            "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
            "rustiq_method": entangling_count(circuit)
        },
        "test_paulis_file": f"test_paulis_{i}.json"
    }
    print(result)
    results.append(result)
    
#     # Save test_paulis to a separate JSON file
#     with open(f'benchmarks/test_paulis_{i}.json', 'w') as paulis_file:
#         json.dump(test_paulis, paulis_file, indent=4)

# # Write results to a JSON file
# with open('benchmarks/UCCSD_results.json', 'w') as json_file:
#     json.dump(results, json_file, indent=4)

print("Results saved to results.json and test_paulis files")



{'electrons': 2, 'orbitals': 4, 'num_paulis': 24, 'times': {'our_time': 0.19781208038330078, 'combined_time': 0.18777704238891602, 'qiskit_time': 0.172454833984375, 'rustiq_time': 0.0003898143768310547}, 'gate_counts': {'our_method': 30, 'combined_method': 27, 'qiskit_method': 41, 'rustiq_method': 26}, 'test_paulis_file': 'test_paulis_0.json'}
{'electrons': 2, 'orbitals': 6, 'num_paulis': 80, 'times': {'our_time': 0.7132539749145508, 'combined_time': 0.9797189235687256, 'qiskit_time': 0.5909130573272705, 'rustiq_time': 0.003512859344482422}, 'gate_counts': {'our_method': 118, 'combined_method': 108, 'qiskit_method': 181, 'rustiq_method': 141}, 'test_paulis_file': 'test_paulis_1.json'}
{'electrons': 4, 'orbitals': 8, 'num_paulis': 320, 'times': {'our_time': 2.957858085632324, 'combined_time': 3.821397066116333, 'qiskit_time': 2.287431001663208, 'rustiq_time': 0.10171794891357422}, 'gate_counts': {'our_method': 561, 'combined_method': 514, 'qiskit_method': 1003, 'rustiq_method': 759}, 't

In [10]:


import json
import time
from qiskit import transpile
from utils import generate_pauli_strings

results = []

for num_paulis in range(20, 100, 10):
    test_paulis = generate_pauli_strings(num_qubits = 20, num_pauli_strings = num_paulis)
    test_params = [0.01 * i for i in range(len(test_paulis))]
    
    # Measure time for our method
    start_time = time.time()
    opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=5)
    end_time = time.time()
    our_time = end_time - start_time

    start_time = time.time()
    opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=5)
    opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    end_time = time.time()
    combined_time = end_time - start_time
    
    # Measure time for Qiskit method
    start_time = time.time()
    origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
    origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    end_time = time.time()
    qiskit_time = end_time - start_time
    
    # Measure time for RustiQ method
    start_time = time.time()
    circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True)
    end_time = time.time()
    rustiq_time = end_time - start_time

    # Measure time for RustiQ method
    start_time = time.time()
    circuit2 = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
    end_time = time.time()
    rustiq_time2 = end_time - start_time
    
    # Collect results
    result = {
        "num_paulis": len(test_paulis),
        "times": {
            "our_time": our_time,
            "combined_time": combined_time,
            "qiskit_time": qiskit_time,
            "rustiq_time": rustiq_time,
            "rustiq_time2": rustiq_time2
        },
        "gate_counts": {
            "our_method": opt_qc_f.count_ops().get('cx', 0),
            "combined_method": opt_qiskit.count_ops().get('cx', 0),
            "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
            "rustiq_method": entangling_count(circuit),
            "rustiq_method_clifford": entangling_count(circuit2)
        },
    }
    print(result)
    results.append(result)
    
#     # Save test_paulis to a separate JSON file
#     with open(f'benchmarks/test_paulis_{i}.json', 'w') as paulis_file:
#         json.dump(test_paulis, paulis_file, indent=4)

# # Write results to a JSON file
# with open('benchmarks/UCCSD_results.json', 'w') as json_file:
#     json.dump(results, json_file, indent=4)

print("Results saved to results.json and test_paulis files")



{'num_paulis': 20, 'times': {'our_time': 0.42133140563964844, 'combined_time': 0.4877188205718994, 'qiskit_time': 1.4230778217315674, 'rustiq_time': 0.01303720474243164, 'rustiq_time2': 0.3015608787536621}, 'gate_counts': {'our_method': 190, 'combined_method': 190, 'qiskit_method': 506, 'rustiq_method': 230, 'rustiq_method_clifford': 404}}
{'num_paulis': 30, 'times': {'our_time': 0.5170979499816895, 'combined_time': 0.68349289894104, 'qiskit_time': 1.1285090446472168, 'rustiq_time': 0.01606011390686035, 'rustiq_time2': 0.3116617202758789}, 'gate_counts': {'our_method': 282, 'combined_method': 282, 'qiskit_method': 790, 'rustiq_method': 345, 'rustiq_method_clifford': 521}}
{'num_paulis': 40, 'times': {'our_time': 0.8180489540100098, 'combined_time': 1.3771209716796875, 'qiskit_time': 1.4820301532745361, 'rustiq_time': 0.016466856002807617, 'rustiq_time2': 0.31891584396362305}, 'gate_counts': {'our_method': 376, 'combined_method': 376, 'qiskit_method': 1080, 'rustiq_method': 404, 'rustiq

In [5]:
#Compare Hamiltonian simulation:
import os, json

# Specify the folder containing the JSON files
folder_path = "benchmarks/HS_paulis"

results = []
# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a JSON file
    if filename.endswith(".json"):
        # Print the filename
        print(filename)
        with open(folder_path + '/' + filename, "r") as file:
            paulis = json.load(file)

        # Function to check if a string contains only 'I' characters
        def is_all_identity(pauli):
            return all(char == 'I' for char in pauli)

        # Filter the list to remove all identity Paulis
        test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
        test_params = [0.01 * i for i in range(len(test_paulis))]

        # Measure time for our method
        start_time = time.time()
        opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=10)
        end_time = time.time()
        our_time = end_time - start_time
        
        # Measure time for Qiskit method
        start_time = time.time()
        origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
        origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        qiskit_time = end_time - start_time

        # Measure time for RustiQ method
        start_time = time.time()
        circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True)
        end_time = time.time()
        rustiq_time = end_time - start_time
        
        # Collect results
        result = {
            "num_paulis": len(test_paulis),
            "times": {
                "our_time": our_time,
                "qiskit_time": qiskit_time,
                "rustiq_time": rustiq_time
            },
            "gate_counts": {
                "our_method": opt_qc_f.count_ops().get('cx', 0),
                "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                "rustiq_method": entangling_count(circuit)
            },
            "test_paulis_file": f'benchmarks/test_' + filename
        }
        print(result)
        results.append(result)

        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/test_' + filename, 'w') as paulis_file:
            json.dump(test_paulis, paulis_file, indent=4)
        
# Write results to a JSON file
with open('benchmarks/Hamiltonian_sim_results.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)
    


LiH.json
{'num_paulis': 61, 'times': {'our_time': 0.6329898834228516, 'qiskit_time': 0.28888511657714844, 'rustiq_time': 0.002214193344116211}, 'gate_counts': {'our_method': 75, 'qiskit_method': 180, 'rustiq_method': 94}, 'test_paulis_file': 'benchmarks/test_LiH.json'}
H2O.json
{'num_paulis': 184, 'times': {'our_time': 2.8661441802978516, 'qiskit_time': 1.414182186126709, 'rustiq_time': 0.02150893211364746}, 'gate_counts': {'our_method': 309, 'qiskit_method': 786, 'rustiq_method': 316}, 'test_paulis_file': 'benchmarks/test_H2O.json'}
benzene.json


KeyboardInterrupt: 

In [62]:
#Compare Hamiltonian simulation:
import os, json

# Specify the folder containing the JSON files
folder_path = "benchmarks/HS_paulis"

results = []
# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a JSON file
    if filename.endswith(".json"):
        # Print the filename
        print(filename)
        with open(folder_path + '/' + filename, "r") as file:
            paulis = json.load(file)

        # Function to check if a string contains only 'I' characters
        def is_all_identity(pauli):
            return all(char == 'I' for char in pauli)

        # Filter the list to remove all identity Paulis
        test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
        test_params = [0.01 * i for i in range(len(test_paulis))]

        # Measure time for our method
        start_time = time.time()
        opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=10)
        end_time = time.time()
        our_time = end_time - start_time
        
        # Measure time for Qiskit method
        start_time = time.time()
        origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
        origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        qiskit_time = end_time - start_time

        # Measure time for RustiQ method
        start_time = time.time()
        circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True)
        end_time = time.time()
        rustiq_time = end_time - start_time

        # Measure time for RustiQ method
        start_time = time.time()
        circuit2 = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
        end_time = time.time()
        rustiq_time2 = end_time - start_time
    
        
        # Collect results
        result = {
            "num_paulis": len(test_paulis),
            "times": {
                "our_time": our_time,
                "qiskit_time": qiskit_time,
                "rustiq_time": rustiq_time,
                "rustiq2_time": rustiq_time2
            },
            "gate_counts": {
                "our_method": opt_qc_f.count_ops().get('cx', 0),
                "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                "rustiq_method": entangling_count(circuit),
                "rustiq_method_fixed": entangling_count(circuit2)
            },
            "test_paulis_file": f'benchmarks/test_' + filename
        }
        print(result)
        results.append(result)

        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/test_' + filename, 'w') as paulis_file:
            json.dump(test_paulis, paulis_file, indent=4)
        
# Write results to a JSON file
with open('benchmarks/Hamiltonian_sim_results.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)
    


LiH.json
{'num_paulis': 61, 'times': {'our_time': 0.649852991104126, 'qiskit_time': 0.32148027420043945, 'rustiq_time': 0.0017421245574951172, 'rustiq2_time': 0.0072231292724609375}, 'gate_counts': {'our_method': 75, 'qiskit_method': 180, 'rustiq_method': 94, 'rustiq_method_fixed': 113}, 'test_paulis_file': 'benchmarks/test_LiH.json'}
H2O.json
{'num_paulis': 184, 'times': {'our_time': 3.222036838531494, 'qiskit_time': 1.1918118000030518, 'rustiq_time': 0.021578073501586914, 'rustiq2_time': 0.0344698429107666}, 'gate_counts': {'our_method': 309, 'qiskit_method': 786, 'rustiq_method': 316, 'rustiq_method_fixed': 350}, 'test_paulis_file': 'benchmarks/test_H2O.json'}
benzene.json
{'num_paulis': 1254, 'times': {'our_time': 23.2484929561615, 'qiskit_time': 11.53954815864563, 'rustiq_time': 8.100050926208496, 'rustiq2_time': 6.131235837936401}, 'gate_counts': {'our_method': 3556, 'qiskit_method': 7602, 'rustiq_method': 3284, 'rustiq_method_fixed': 3352}, 'test_paulis_file': 'benchmarks/test_b

In [57]:
with open("benchmarks/HS_paulis/benzene.json", "r") as file:
    paulis = json.load(file)

In [63]:
sorted_entanglers_f

[[['IIIIIIIIIIIZ', 0.0]],
 [['IIIIIIIIIIYY', 0.01], ['IIIIIIIIIIZX', -0.02]],
 [['IIIIIIIIIYXI', 0.03], ['IIIIIIIIIYIY', -0.04]],
 [['IIIIIIIIXYII', -0.06], ['IIIIIIIIYIYZ', 0.05]],
 [['IIIIIIIYYIII', -0.07], ['IIIIIIIYIYZI', 0.08]],
 [['IIIIIIIIIIIY', 0.11], ['IIIIIIXXIIYI', 0.1], ['IIIIIIYIYZII', -0.09]],
 [['IIIIIIIIIIXY', -0.12], ['IIIIIIIZIXIY', 0.13]],
 [['IIIIIIIIZYII', 0.15], ['IIIIIIIZIXYZ', -0.14]],
 [['IIIIIIIIZYII', 0.16], ['IIIIIIIZYIZI', -0.17]],
 [['IIIIIIIIIIIY', -0.2], ['IIIIIIIYXIYI', -0.19], ['IIIIIIYIXYII', 0.18]],
 [['IIIIIIIIIXIX', -0.21], ['IIIIIIZXZIIY', -0.22]],
 [['IIIIIIIIIYII', 0.23], ['IIIIIIXXYIZX', 0.24]],
 [['IIIIIIIIIZIY', 0.27], ['IIIIIIIIYYII', -0.25], ['IIIIIIYXIIYI', 0.26]],
 [['IIIIIIIIZZIY', 0.28], ['IIIIIIZZZZIX', 0.29]],
 [['IIIIIIZIXIII', -0.32], ['IIIIIIXIZZII', 0.31], ['IIIIIIZZZXIZ', -0.3]],
 [['IIIIIIZIXIII', 0.33], ['IIIIIIXZIIIZ', -0.34]],
 [['IIIIIZIIIIII', 0.36], ['IIIIIIYIYIIZ', 0.35000000000000003]],
 [['IIIIYYIIIIII', 0.37], ['IIIIZX

In [58]:
paulis

['IIIIIIIIIIII',
 'IIIIIIIIIIIZ',
 'IIIIIIIIIIYY',
 'IIIIIIIIIIXX',
 'IIIIIIIIIYZY',
 'IIIIIIIIIXZX',
 'IIIIIIIIYZZY',
 'IIIIIIIIXZZX',
 'IIIIIIIYZZZY',
 'IIIIIIIXZZZX',
 'IIIIIIYZZZZY',
 'IIIIIIXZZZZX',
 'IIIIIIIIIIZI',
 'IIIIIIIIIYYI',
 'IIIIIIIIIXXI',
 'IIIIIIIIYZYI',
 'IIIIIIIIXZXI',
 'IIIIIIIYZZYI',
 'IIIIIIIXZZXI',
 'IIIIIIYZZZYI',
 'IIIIIIXZZZXI',
 'IIIIIIIIIZII',
 'IIIIIIIIYYII',
 'IIIIIIIIXXII',
 'IIIIIIIYZYII',
 'IIIIIIIXZXII',
 'IIIIIIYZZYII',
 'IIIIIIXZZXII',
 'IIIIIIIIZIII',
 'IIIIIIIYYIII',
 'IIIIIIIXXIII',
 'IIIIIIYZYIII',
 'IIIIIIXZXIII',
 'IIIIIIIZIIII',
 'IIIIIIYYIIII',
 'IIIIIIXXIIII',
 'IIIIIIZIIIII',
 'IIIIIZIIIIII',
 'IIIIYYIIIIII',
 'IIIIXXIIIIII',
 'IIIYZYIIIIII',
 'IIIXZXIIIIII',
 'IIYZZYIIIIII',
 'IIXZZXIIIIII',
 'IYZZZYIIIIII',
 'IXZZZXIIIIII',
 'YZZZZYIIIIII',
 'XZZZZXIIIIII',
 'IIIIZIIIIIII',
 'IIIYYIIIIIII',
 'IIIXXIIIIIII',
 'IIYZYIIIIIII',
 'IIXZXIIIIIII',
 'IYZZYIIIIIII',
 'IXZZXIIIIIII',
 'YZZZYIIIIIII',
 'XZZZXIIIIIII',
 'IIIZIIIIIIII',
 'IIYYIIIIIIII

In [4]:
#new lookahead

#Compare Hamiltonian simulation:
import os, json

# Specify the folder containing the JSON files
folder_path = "benchmarks/HS_paulis"

results = []
# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a JSON file
    if filename.endswith(".json"):
        # Print the filename
        print(filename)
        with open(folder_path + '/' + filename, "r") as file:
            paulis = json.load(file)

        # Function to check if a string contains only 'I' characters
        def is_all_identity(pauli):
            return all(char == 'I' for char in pauli)

        # Filter the list to remove all identity Paulis
        test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
        test_params = [0.01 * i for i in range(len(test_paulis))]

        # Measure time for our method
        start_time = time.time()
        opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=10)
        end_time = time.time()
        our_time = end_time - start_time
        
        # Measure time for Qiskit method
        start_time = time.time()
        origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
        origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        qiskit_time = end_time - start_time

        # Measure time for RustiQ method
        start_time = time.time()
        circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True)
        end_time = time.time()
        rustiq_time = end_time - start_time
        
        # Collect results
        result = {
            "num_paulis": len(test_paulis),
            "times": {
                "our_time": our_time,
                "qiskit_time": qiskit_time,
                "rustiq_time": rustiq_time
            },
            "gate_counts": {
                "our_method": opt_qc_f.count_ops().get('cx', 0),
                "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                "rustiq_method": entangling_count(circuit)
            },
            "test_paulis_file": f'benchmarks/test_' + filename
        }
        print(result)
        results.append(result)

        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/test_' + filename, 'w') as paulis_file:
            json.dump(test_paulis, paulis_file, indent=4)
        
# Write results to a JSON file
with open('benchmarks/Hamiltonian_sim_results.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)
    


LiH.json
{'num_paulis': 61, 'times': {'our_time': 0.8232619762420654, 'qiskit_time': 1.1564009189605713, 'rustiq_time': 0.001741170883178711}, 'gate_counts': {'our_method': 75, 'qiskit_method': 180, 'rustiq_method': 94}, 'test_paulis_file': 'benchmarks/test_LiH.json'}
H2O.json
{'num_paulis': 184, 'times': {'our_time': 2.8913609981536865, 'qiskit_time': 1.2841732501983643, 'rustiq_time': 0.02041172981262207}, 'gate_counts': {'our_method': 309, 'qiskit_method': 786, 'rustiq_method': 316}, 'test_paulis_file': 'benchmarks/test_H2O.json'}
benzene.json
{'num_paulis': 1254, 'times': {'our_time': 25.46829628944397, 'qiskit_time': 10.438604831695557, 'rustiq_time': 4.942078113555908}, 'gate_counts': {'our_method': 3556, 'qiskit_method': 7602, 'rustiq_method': 3284}, 'test_paulis_file': 'benchmarks/test_benzene.json'}


In [ ]:
#new lookahead

#Compare Hamiltonian simulation:
import os, json

# Specify the folder containing the JSON files
folder_path = "benchmarks/HS_paulis"

results = []
# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a JSON file
    if filename.endswith(".json"):
        # Print the filename
        print(filename)
        with open(folder_path + '/' + filename, "r") as file:
            paulis = json.load(file)

        # Function to check if a string contains only 'I' characters
        def is_all_identity(pauli):
            return all(char == 'I' for char in pauli)

        # Filter the list to remove all identity Paulis
        test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
        test_params = [0.01 * i for i in range(len(test_paulis))]

        # Measure time for our method
        start_time = time.time()
        opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=5)
        end_time = time.time()
        our_time = end_time - start_time
        
        # Measure time for Qiskit method
        start_time = time.time()
        origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
        origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        qiskit_time = end_time - start_time

        # Measure time for RustiQ method
        start_time = time.time()
        circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True)
        end_time = time.time()
        rustiq_time = end_time - start_time
        
        # Collect results
        result = {
            "num_paulis": len(test_paulis),
            "times": {
                "our_time": our_time,
                "qiskit_time": qiskit_time,
                "rustiq_time": rustiq_time
            },
            "gate_counts": {
                "our_method": opt_qc_f.count_ops().get('cx', 0),
                "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                "rustiq_method": entangling_count(circuit)
            },
            "test_paulis_file": f'benchmarks/test_' + filename
        }
        print(result)
        results.append(result)

        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/test_' + filename, 'w') as paulis_file:
            json.dump(test_paulis, paulis_file, indent=4)
        
# Write results to a JSON file
with open('benchmarks/Hamiltonian_sim_results.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)
    


In [ ]:
max_cutrun_experiment_folder(folder_path = None, filename = None, save_output = False, lookahead_size = 5):

In [40]:
#Compare Hamiltonian simulation:
import os, json
def run_experiment_folder(folder_path = None, filename = None, save_output = False, lookahead_size = 5):

    results = []
    if filename == None:
        file_list = os.listdir(folder_path)
    else:
        file_list = [filename]
    # Iterate over all files in the folder
    for filename in file_list:
        # Check if the file is a JSON file
        if filename.endswith(".json"):
            # Print the filename
            print(filename)
            with open(folder_path + '/' + filename, "r") as file:
                paulis = json.load(file)

            # Function to check if a string contains only 'I' characters
            def is_all_identity(pauli):
                return all(char == 'I' for char in pauli)

            # Filter the list to remove all identity Paulis
            test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
            test_params = [0.01 * i for i in range(len(test_paulis))]

            # Measure time for our method
            start_time = time.time()
            opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=lookahead_size)
            end_time = time.time()
            our_time = end_time - start_time

        start_time = time.time()
        opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=lookahead_size)
        opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        combined_time = end_time - start_time
        
        # Measure time for Qiskit method
        start_time = time.time()
        origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
        origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        qiskit_time = end_time - start_time

        # Measure time for RustiQ method
        start_time = time.time()
        circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True)
        end_time = time.time()
        rustiq_time = end_time - start_time
        
        # Collect results
        result = {
            "num_paulis": len(test_paulis),
            "times": {
                "our_time": our_time,
                "combined_time": combined_time,
                "qiskit_time": qiskit_time,
                "rustiq_time": rustiq_time
            },
            "gate_counts": {
                "our_method": opt_qc_f.count_ops().get('cx', 0),
                "combined_method": opt_qiskit.count_ops().get('cx', 0),
                "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                "rustiq_method": entangling_count(circuit)
            },
            "test_paulis_file": f'benchmarks/test_' + filename
        }
        print(result)
        results.append(result)
        if save_output == True:
            # Save test_paulis to a separate JSON file
            with open(f'benchmarks/test_' + filename, 'w') as paulis_file:
                json.dump(test_paulis, paulis_file, indent=4)
    if save_output == True:  
        # Write results to a JSON file
        with open('benchmarks/LABS_results.json', 'w') as json_file:
            json.dump(results, json_file, indent=4)
    

    

In [51]:
#Compare Hamiltonian simulation:
import os, json
def run_experiment_paulis(test_paulis, test_params = None, save_output = False, lookahead_size = 5, filename = "Paulis"):

    results = []

    def is_all_identity(pauli):
        return all(char == 'I' for char in pauli)
    paulis = test_paulis
    # Filter the list to remove all identity Paulis
    test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
    if test_params is None:
        test_params = [0.01 * i for i in range(len(test_paulis))]

    # Measure time for our method
    start_time = time.time()
    opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=lookahead_size)
    end_time = time.time()
    our_time = end_time - start_time

    start_time = time.time()
    opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=lookahead_size)
    opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    end_time = time.time()
    combined_time = end_time - start_time
    
    # Measure time for Qiskit method
    start_time = time.time()
    origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
    origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    end_time = time.time()
    qiskit_time = end_time - start_time

    # Measure time for RustiQ method
    start_time = time.time()
    circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True)
    end_time = time.time()
    rustiq_time = end_time - start_time
    
    # Collect results
    result = {
        "num_paulis": len(test_paulis),
        "times": {
            "our_time": our_time,
            "combined_time": combined_time,
            "qiskit_time": qiskit_time,
            "rustiq_time": rustiq_time
        },
        "gate_counts": {
            "our_method": opt_qc_f.count_ops().get('cx', 0),
            "combined_method": opt_qiskit.count_ops().get('cx', 0),
            "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
            "rustiq_method": entangling_count(circuit)
        },
        "test_paulis_file": f'benchmarks/test_' + filename
    }
    print(result)
    results.append(result)
    if save_output == True:
        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/test_' + filename, 'w') as paulis_file:
            json.dump(test_paulis, paulis_file, indent=4)
    return sorted_entanglers_f
    

    

In [54]:
test_paulis = generate_UCCSD_entanglers(6, 12)
entanglers = run_experiment_paulis(test_paulis = test_paulis, save_output = False, lookahead_size = 5)

{'num_paulis': 1656, 'times': {'our_time': 23.276309967041016, 'combined_time': 23.03163981437683, 'qiskit_time': 18.522618055343628, 'rustiq_time': 22.582128047943115}, 'gate_counts': {'our_method': 3264, 'combined_method': 2999, 'qiskit_method': 5723, 'rustiq_method': 4644}, 'test_paulis_file': 'benchmarks/test_Paulis'}


In [55]:
entanglers

[[['YZZZZZXIIIII', 0.0]],
 [['YIIIIIIIIIII', 0.01], ['ZIIIIIXIIIII', -0.02]],
 [['XIIIIIZIIIII', 0.03], ['YZIIIIIZXIII', -0.04]],
 [['IIIIIIIIYIII', 0.06], ['YZIIIIYIZIII', 0.05]],
 [['ZIIIIIIIXIII', 0.07], ['YIIIIIIIIZXI', -0.08]],
 [['IIIIIIIIIIYI', 0.1], ['XIIIIIIIYIZI', -0.09]],
 [['IZIIIIIYIIII', 0.12], ['YIIIIIIIXIYI', 0.11]],
 [['IIIIIIIYIIII', -0.14], ['IZZIIIIZIIII', 0.13]],
 [['IZIIIIIXIIII', 0.15], ['IXZIIIYIIXII', -0.16]],
 [['IIIIIIIIIYII', 0.18], ['IXIIIIIYIZII', 0.17]],
 [['IZIIIIIIIXII', 0.19], ['IYIIIIIIXIIX', -0.2]],
 [['IIIIIIIIIIIY', 0.22], ['IXIIIIIIIYIZ', -0.21]],
 [['IIZIIIXIIIII', -0.24], ['IYIIIIIIIXIY', 0.23]],
 [['IIIIIIYIIIII', -0.26], ['IIZZIIZIIIII', -0.25]],
 [['IIZIIIXIIIII', 0.27], ['IIXZIIIZXIZI', 0.28]],
 [['IIIIIIIIIIYI', 0.3], ['IIXIIIYIIIZI', -0.29]],
 [['IIZIIIIIIIXI', 0.31], ['ZIYIIIIIIIII', -0.32]],
 [['IIYIIIIIIIYI', -0.33], ['YIZIIIIIIIII', 0.34]],
 [['IIIZIIIYIIII', 0.36], ['YIXIIIIIIIYI', 0.35000000000000003]],
 [['IIIIIIIYZIII', 0.38], ['II

In [ ]:
test_paulis = generate_UCCSD_entanglers(6, 12)
entanglers = run_experiment_paulis(test_paulis = test_paulis, save_output = False, lookahead_size = 5)

In [ ]:
test_paulis = generate_UCCSD_entanglers(4, 8)
entanglers = run_experiment_paulis(test_paulis = test_paulis, save_output = False, lookahead_size = 5)

In [ ]:
entanglers

In [36]:
test_paulis = ['ZZZZZZZZZZZZ', 'XXXXXXYYYYYY']
test_params = [0.01 * i for i in range(len(test_paulis))]

In [37]:
opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=5)
 

In [38]:
opt_qc_f.count_ops()

OrderedDict([('cx', 17), ('h', 6), ('rz', 2)])

In [39]:
sorted_entanglers_f

[[['ZZZZZZZZZZZZ', 0.0], ['IXIXIXZXIXIX', -0.01]]]

In [31]:
circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True)
entangling_count(circuit)


7

In [20]:
run_experiment_folder(folder_path = "benchmarks/HS_paulis", save_output = False, lookahead_size = 5)

LiH.json
{'num_paulis': 61, 'times': {'our_time': 0.48439598083496094, 'combined_time': 0.9360213279724121, 'qiskit_time': 0.28406810760498047, 'rustiq_time': 0.0017552375793457031}, 'gate_counts': {'our_method': 78, 'combined_method': 76, 'qiskit_method': 180, 'rustiq_method': 94}, 'test_paulis_file': 'benchmarks/test_LiH.json'}
H2O.json
{'num_paulis': 184, 'times': {'our_time': 1.8548388481140137, 'combined_time': 2.200809955596924, 'qiskit_time': 1.4125192165374756, 'rustiq_time': 0.022082090377807617}, 'gate_counts': {'our_method': 338, 'combined_method': 335, 'qiskit_method': 786, 'rustiq_method': 316}, 'test_paulis_file': 'benchmarks/test_H2O.json'}
benzene.json
{'num_paulis': 1254, 'times': {'our_time': 15.784124851226807, 'combined_time': 20.428149938583374, 'qiskit_time': 13.283131122589111, 'rustiq_time': 7.251043081283569}, 'gate_counts': {'our_method': 3445, 'combined_method': 3436, 'qiskit_method': 7602, 'rustiq_method': 3284}, 'test_paulis_file': 'benchmarks/test_benzene.

In [12]:
#Compare Hamiltonian simulation:
import os, json

# Specify the folder containing the JSON files
folder_path = "benchmarks/labs_paulis"

results = []
# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a JSON file
    if filename.endswith(".json"):
        # Print the filename
        print(filename)
        with open(folder_path + '/' + filename, "r") as file:
            paulis = json.load(file)
        paulis = paulis[0]
        # Function to check if a string contains only 'I' characters
        def is_all_identity(pauli):
            return all(char == 'I' for char in pauli)

        # Filter the list to remove all identity Paulis
        test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)] * 3
        test_params = [0.01 * i for i in range(len(test_paulis))]

        # Measure time for our method
        start_time = time.time()
        opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=20)
        end_time = time.time()
        our_time = end_time - start_time

        start_time = time.time()
        opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=len(test_paulis)/6)
        opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        combined_time = end_time - start_time
        
        # Measure time for Qiskit method
        start_time = time.time()
        origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
        origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        qiskit_time = end_time - start_time

        # Measure time for RustiQ method
        start_time = time.time()
        circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True)
        end_time = time.time()
        rustiq_time = end_time - start_time
        
        # Collect results
        result = {
            "num_paulis": len(test_paulis),
            "times": {
                "our_time": our_time,
                "combined_time": combined_time,
                "qiskit_time": qiskit_time,
                "rustiq_time": rustiq_time
            },
            "gate_counts": {
                "our_method": opt_qc_f.count_ops().get('cx', 0),
                "combined_method": opt_qiskit.count_ops().get('cx', 0),
                "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                "rustiq_method": entangling_count(circuit)
            },
            "test_paulis_file": f'benchmarks/test_' + filename
        }
        print(result)
        results.append(result)

#         # Save test_paulis to a separate JSON file
#         with open(f'benchmarks/test_' + filename, 'w') as paulis_file:
#             json.dump(test_paulis, paulis_file, indent=4)
        
# # Write results to a JSON file
# with open('benchmarks/LABS_results.json', 'w') as json_file:
#     json.dump(results, json_file, indent=4)
    


labs_n14_layers1.json
{'num_paulis': 609, 'times': {'our_time': 279.3390016555786, 'combined_time': 3455.8212690353394, 'qiskit_time': 1.36922287940979, 'rustiq_time': 0.14318418502807617}, 'gate_counts': {'our_method': 284, 'combined_method': 281, 'qiskit_method': 2778, 'rustiq_method': 266}, 'test_paulis_file': 'benchmarks/test_labs_n14_layers1.json'}
labs_n13_layers1.json
{'num_paulis': 483, 'times': {'our_time': 468.9600100517273, 'combined_time': 261.8826241493225, 'qiskit_time': 0.7865138053894043, 'rustiq_time': 0.0845339298248291}, 'gate_counts': {'our_method': 210, 'combined_method': 208, 'qiskit_method': 2160, 'rustiq_method': 215}, 'test_paulis_file': 'benchmarks/test_labs_n13_layers1.json'}
labs_n12_layers1.json
{'num_paulis': 375, 'times': {'our_time': 95.03042697906494, 'combined_time': 158.40880298614502, 'qiskit_time': 0.9184660911560059, 'rustiq_time': 0.05136704444885254}, 'gate_counts': {'our_method': 157, 'combined_method': 154, 'qiskit_method': 1742, 'rustiq_method

KeyboardInterrupt: 

In [6]:
#newlookahead
#Compare Hamiltonian simulation:
import os, json

# Specify the folder containing the JSON files
folder_path = "benchmarks/labs_paulis"

results = []
# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a JSON file
    if filename.endswith(".json"):
        # Print the filename
        print(filename)
        with open(folder_path + '/' + filename, "r") as file:
            paulis = json.load(file)
        paulis = paulis[0]
        # Function to check if a string contains only 'I' characters
        def is_all_identity(pauli):
            return all(char == 'I' for char in pauli)

        # Filter the list to remove all identity Paulis
        test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
        test_params = [0.01 * i for i in range(len(test_paulis))]

        # Measure time for our method
        start_time = time.time()
        opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=20)
        end_time = time.time()
        our_time = end_time - start_time

        start_time = time.time()
        opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=20)
        opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        combined_time = end_time - start_time
        
        # Measure time for Qiskit method
        start_time = time.time()
        origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
        origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        qiskit_time = end_time - start_time

        # Measure time for RustiQ method
        start_time = time.time()
        circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True)
        end_time = time.time()
        rustiq_time = end_time - start_time
        
        # Collect results
        result = {
            "num_paulis": len(test_paulis),
            "times": {
                "our_time": our_time,
                "combined_time": combined_time,
                "qiskit_time": qiskit_time,
                "rustiq_time": rustiq_time
            },
            "gate_counts": {
                "our_method": opt_qc_f.count_ops().get('cx', 0),
                "combined_method": opt_qiskit.count_ops().get('cx', 0),
                "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                "rustiq_method": entangling_count(circuit)
            },
            "test_paulis_file": f'benchmarks/test_' + filename
        }
        print(result)
        results.append(result)

        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/test_' + filename, 'w') as paulis_file:
            json.dump(test_paulis, paulis_file, indent=4)
        
# Write results to a JSON file
with open('benchmarks/LABS_results.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)
    


labs_n14_layers1.json
{'num_paulis': 203, 'times': {'our_time': 40.66877222061157, 'combined_time': 35.72797608375549, 'qiskit_time': 0.3453359603881836, 'rustiq_time': 0.028088092803955078}, 'gate_counts': {'our_method': 263, 'combined_method': 263, 'qiskit_method': 922, 'rustiq_method': 278}, 'test_paulis_file': 'benchmarks/test_labs_n14_layers1.json'}
labs_n13_layers1.json
{'num_paulis': 161, 'times': {'our_time': 21.943038940429688, 'combined_time': 25.33715295791626, 'qiskit_time': 0.2622668743133545, 'rustiq_time': 0.01676321029663086}, 'gate_counts': {'our_method': 210, 'combined_method': 210, 'qiskit_method': 716, 'rustiq_method': 211}, 'test_paulis_file': 'benchmarks/test_labs_n13_layers1.json'}
labs_n12_layers1.json
{'num_paulis': 125, 'times': {'our_time': 13.450691938400269, 'combined_time': 13.013431072235107, 'qiskit_time': 0.19855904579162598, 'rustiq_time': 0.010106801986694336}, 'gate_counts': {'our_method': 150, 'combined_method': 150, 'qiskit_method': 578, 'rustiq_me

KeyboardInterrupt: 

In [1]:
#newlookahead
#Compare Hamiltonian simulation:
import os, json

# Specify the folder containing the JSON files
folder_path = "benchmarks/labs_paulis"

results = []
# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a JSON file
    if filename.endswith(".json"):
        # Print the filename
        print(filename)
        with open(folder_path + '/' + filename, "r") as file:
            paulis = json.load(file)
        paulis = paulis[0]
        # Function to check if a string contains only 'I' characters
        def is_all_identity(pauli):
            return all(char == 'I' for char in pauli)

        # Filter the list to remove all identity Paulis
        test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
        test_params = [0.01 * i for i in range(len(test_paulis))]

        # Measure time for our method
        start_time = time.time()
        opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=5)
        end_time = time.time()
        our_time = end_time - start_time

        start_time = time.time()
        opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=5)
        opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        combined_time = end_time - start_time
        
        # Measure time for Qiskit method
        start_time = time.time()
        origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
        origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        qiskit_time = end_time - start_time

        # Measure time for RustiQ method
        start_time = time.time()
        circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True)
        end_time = time.time()
        rustiq_time = end_time - start_time
        
        # Collect results
        result = {
            "num_paulis": len(test_paulis),
            "times": {
                "our_time": our_time,
                "combined_time": combined_time,
                "qiskit_time": qiskit_time,
                "rustiq_time": rustiq_time
            },
            "gate_counts": {
                "our_method": opt_qc_f.count_ops().get('cx', 0),
                "combined_method": opt_qiskit.count_ops().get('cx', 0),
                "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                "rustiq_method": entangling_count(circuit)
            },
            "test_paulis_file": f'benchmarks/test_' + filename
        }
        print(result)
        results.append(result)

        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/test_' + filename, 'w') as paulis_file:
            json.dump(test_paulis, paulis_file, indent=4)
        
# Write results to a JSON file
with open('benchmarks/LABS_results.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)
    


labs_n14_layers1.json


NameError: name 'time' is not defined

In [4]:
#newlookahead
#Compare Hamiltonian simulation:
import os, json

# Specify the folder containing the JSON files
folder_path = "benchmarks/max_cut_paulis"

results = []
# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a JSON file
    if filename.endswith(".json"):
        # Print the filename
        print(filename)
        with open(folder_path + '/' + filename, "r") as file:
            paulis = json.load(file)
        #expand them
        flattened_paulis = [item for sublist in paulis for item in sublist]
        # Function to check if a string contains only 'I' characters
        def is_all_identity(pauli):
            return all(char == 'I' for char in pauli)

        # Filter the list to remove all identity Paulis
        length = len(flattened_paulis[0])
        test_paulis = [pauli for pauli in flattened_paulis if not is_all_identity(pauli)] 
        test_params = [0.01 * i for i in range(len(test_paulis))]

        # Measure time for our method
        start_time = time.time()
        opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=5)
        end_time = time.time()
        our_time = end_time - start_time

        start_time = time.time()
        opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=5)
        opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        combined_time = end_time - start_time
        
        # Measure time for Qiskit method
        start_time = time.time()
        origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
        origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        qiskit_time = end_time - start_time

        # Measure time for RustiQ method
        start_time = time.time()
        circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True)
        end_time = time.time()
        rustiq_time = end_time - start_time
        
        # Collect results
        result = {
            "num_paulis": len(test_paulis),
            "times": {
                "our_time": our_time,
                "combined_time": combined_time,
                "qiskit_time": qiskit_time,
                "rustiq_time": rustiq_time
            },
            "gate_counts": {
                "our_method": opt_qc_f.count_ops().get('cx', 0),
                "combined_method": opt_qiskit.count_ops().get('cx', 0),
                "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                "rustiq_method": entangling_count(circuit)
            },
            "test_paulis_file": f'benchmarks/test_' + filename
        }
        print(result)
        results.append(result)

        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/test_' + filename, 'w') as paulis_file:
            json.dump(test_paulis, paulis_file, indent=4)
        
# Write results to a JSON file
with open('benchmarks/maxcut_results.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)
    


max_cut_benchmark_n10_e20.json
{'num_paulis': 150, 'times': {'our_time': 2.91294002532959, 'combined_time': 2.803765058517456, 'qiskit_time': 0.3244941234588623, 'rustiq_time': 0.008272886276245117}, 'gate_counts': {'our_method': 179, 'combined_method': 179, 'qiskit_method': 198, 'rustiq_method': 148}, 'test_paulis_file': 'benchmarks/test_max_cut_benchmark_n10_e20.json'}
max_cut_benchmark_n5_e20.json
{'num_paulis': 150, 'times': {'our_time': 1.1720380783081055, 'combined_time': 1.428692102432251, 'qiskit_time': 0.1905529499053955, 'rustiq_time': 0.0060541629791259766}, 'gate_counts': {'our_method': 114, 'combined_method': 114, 'qiskit_method': 198, 'rustiq_method': 129}, 'test_paulis_file': 'benchmarks/test_max_cut_benchmark_n5_e20.json'}
max_cut_benchmark_n15_e70.json


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f8a001164d0>>
Traceback (most recent call last):
  File "/Users/revilooliver/Documents/quantum_computing/anaconda/anaconda3/envs/qiskit1.0/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [4]:
#newlookahead
#Compare Hamiltonian simulation:
import os, json

# Specify the folder containing the JSON files
folder_path = "benchmarks/max_cut_paulis"

results = []
# Iterate over all files in the folder
for filename in ["max_cut_benchmark_n10_e22_l10.json"]:#os.listdir(folder_path): #["max_cut_benchmark_n10_e20.json"]
    # Check if the file is a JSON file
    if filename.endswith(".json"):
        # Print the filename
        print(filename)
        with open(folder_path + '/' + filename, "r") as file:
            paulis = json.load(file)
        #expand them
        # flattened_paulis = [item for sublist in paulis for item in sublist]
        # Function to check if a string contains only 'I' characters
        def is_all_identity(pauli):
            return all(char == 'I' for char in pauli)

        # Filter the list to remove all identity Paulis
        length = len(paulis)
        test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)] 
        test_params = [0.01 * i for i in range(len(test_paulis))]

        # Measure time for our method
        start_time = time.time()
        opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=10)
        end_time = time.time()
        our_time = end_time - start_time

        start_time = time.time()
        opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=10)
        opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        combined_time = end_time - start_time
        
        # Measure time for our method
        start_time = time.time()
        opt_qc_f3, append_clifford_f3, sorted_entanglers_f3 = fc_tree_sort_lookahead_estimate(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=10)
        end_time = time.time()
        our_time3 = end_time - start_time

        # Measure time for Qiskit method
        start_time = time.time()
        origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
        origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        qiskit_time = end_time - start_time

        # Measure time for RustiQ method
        start_time = time.time()
        circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
        end_time = time.time()
        rustiq_time = end_time - start_time
        
        # Collect results
        result = {
            "num_paulis": len(test_paulis),
            "times": {
                "our_time": our_time,
                "combined_time": combined_time,
                "sort_time": our_time3,
                "qiskit_time": qiskit_time,
                "rustiq_time": rustiq_time
            },
            "gate_counts": {
                "our_method": opt_qc_f.count_ops().get('cx', 0),
                "sort_method": opt_qc_f3.count_ops().get('cx', 0),
                "combined_method": opt_qiskit.count_ops().get('cx', 0),
                "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                "rustiq_method": entangling_count(circuit)
            },
            "test_paulis_file": f'benchmarks/test_' + filename
        }
        print(result)
        results.append(result)

        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/test_' + filename, 'w') as paulis_file:
            json.dump(test_paulis, paulis_file, indent=4)
        
# Write results to a JSON file
with open('benchmarks/maxcut_results.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)
    


max_cut_benchmark_n10_e22_l10.json
{'num_paulis': 320, 'times': {'our_time': 7.376744985580444, 'combined_time': 7.682384967803955, 'sort_time': 5.282937288284302, 'qiskit_time': 0.4143819808959961, 'rustiq_time': 0.05238795280456543}, 'gate_counts': {'our_method': 371, 'sort_method': 344, 'combined_method': 369, 'qiskit_method': 418, 'rustiq_method': 360}, 'test_paulis_file': 'benchmarks/test_max_cut_benchmark_n10_e22_l10.json'}


In [5]:
sorted_entanglers_f2

[[['IIIIIIZIZI', 0.0],
  ['ZIIZIIIIII', 0.03],
  ['IZIIIZIIII', 0.07],
  ['IZIIIIIIZI', 0.1],
  ['ZZIZIIIIII', 0.11],
  ['ZIZIIIIIII', 0.02],
  ['IZZIIIIIII', 0.17],
  ['ZIIIIIIIIZ', 0.05],
  ['IIIIIIIIIZ', 0.06],
  ['IZIIIIZIZI', 0.08],
  ['IIZIIIZIII', 0.01],
  ['IZIIIIZIII', 0.13],
  ['ZIZIIIIZII', 0.09],
  ['ZIIIZIIIII', 0.16],
  ['ZIZIIIIIIZ', 0.21],
  ['IIIIIIZIIZ', 0.14],
  ['IZIIIIIIIZ', 0.18],
  ['IIIIIZIIIZ', 0.12],
  ['IIZIIZZIII', 0.15],
  ['IIIIIZIZIZ', 0.19],
  ['IZIIIZIIZI', 0.04],
  ['IZZZIIIIIZ', 0.2]],
 [['IIIXIIIIII', 0.22],
  ['IIIIIIXIII', 0.23],
  ['IIIIIIIIXI', 0.24],
  ['IIXIIIIIII', 0.25],
  ['XIIIXIIIII', 0.26],
  ['ZXIIIIIXII', 0.31],
  ['IIIIIIIZIX', 0.27],
  ['IIIIIIZZII', 0.28],
  ['IIIIIXIZII', 0.29],
  ['ZZZZIZIIZZ', 0.3]],
 [['IIXIIIIIII', 0.34],
  ['IXIIIIIIII', 0.46],
  ['IIIIIIXIII', 0.45],
  ['IIIIIXIIII', 0.49],
  ['IZIIIZIIII', 0.5],
  ['IZZIIIIIII', 0.37],
  ['IZIIIIIIII', 0.38],
  ['IZIXIIIIII', 0.35000000000000003],
  ['IZIIIIIIIX', 0.470000000

In [6]:
#newlookahead
#Compare Hamiltonian simulation:
import os, json

# Specify the folder containing the JSON files
folder_path = "benchmarks/max_cut_paulis"

results = []
# Iterate over all files in the folder
for filename in ["max_cut_benchmark_n10_e22_l10.json"]:#os.listdir(folder_path): #["max_cut_benchmark_n10_e20.json"]
    # Check if the file is a JSON file
    if filename.endswith(".json"):
        # Print the filename
        print(filename)
        with open(folder_path + '/' + filename, "r") as file:
            paulis = json.load(file)
        #expand them
        # flattened_paulis = [item for sublist in paulis for item in sublist]
        # Function to check if a string contains only 'I' characters
        def is_all_identity(pauli):
            return all(char == 'I' for char in pauli)

        # Filter the list to remove all identity Paulis
        length = len(paulis)
        test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)] 
        test_params = [0.01 * i for i in range(len(test_paulis))]

        # Measure time for our method
        start_time = time.time()
        opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=10)
        end_time = time.time()
        our_time = end_time - start_time

        start_time = time.time()
        opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=10)
        opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        combined_time = end_time - start_time
        
        # Measure time for our method
        start_time = time.time()
        opt_qc_f3, append_clifford_f3, sorted_entanglers_f3 = fc_tree_sort_lookahead_estimate(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=10)
        end_time = time.time()
        our_time3 = end_time - start_time

        # Measure time for Qiskit method
        start_time = time.time()
        origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
        origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        qiskit_time = end_time - start_time

        # Measure time for RustiQ method
        start_time = time.time()
        circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
        end_time = time.time()
        rustiq_time = end_time - start_time
        
        # Collect results
        result = {
            "num_paulis": len(test_paulis),
            "times": {
                "our_time": our_time,
                "combined_time": combined_time,
                "sort_time": our_time3,
                "qiskit_time": qiskit_time,
                "rustiq_time": rustiq_time
            },
            "gate_counts": {
                "our_method": opt_qc_f.count_ops().get('cx', 0),
                "sort_method": opt_qc_f3.count_ops().get('cx', 0),
                "combined_method": opt_qiskit.count_ops().get('cx', 0),
                "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                "rustiq_method": entangling_count(circuit)
            },
            "test_paulis_file": f'benchmarks/test_' + filename
        }
        print(result)
        results.append(result)

        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/test_' + filename, 'w') as paulis_file:
            json.dump(test_paulis, paulis_file, indent=4)
        
# Write results to a JSON file
with open('benchmarks/maxcut_results.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)
    


max_cut_benchmark_n10_e22_l10.json


In [8]:
#newlookahead
#Compare Hamiltonian simulation:
import os, json

# Specify the folder containing the JSON files
folder_path = "benchmarks/max_cut_paulis"

results = []
# Iterate over all files in the folder
for filename in ["max_cut_benchmark_n10_e45_l20.json"]:#os.listdir(folder_path): #["max_cut_benchmark_n10_e20.json"]
    # Check if the file is a JSON file
    if filename.endswith(".json"):
        # Print the filename
        print(filename)
        with open(folder_path + '/' + filename, "r") as file:
            paulis = json.load(file)
        #expand them
        # flattened_paulis = [item for sublist in paulis for item in sublist]
        # Function to check if a string contains only 'I' characters
        def is_all_identity(pauli):
            return all(char == 'I' for char in pauli)

        # Filter the list to remove all identity Paulis
        length = len(paulis)
        test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)] 
        test_params = [0.01 * i for i in range(len(test_paulis))]

        # Measure time for our method
        start_time = time.time()
        opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=10)
        end_time = time.time()
        our_time = end_time - start_time

        start_time = time.time()
        opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=10)
        opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        combined_time = end_time - start_time
        
        # Measure time for our method
        start_time = time.time()
        opt_qc_f3, append_clifford_f3, sorted_entanglers_f3 = fc_tree_sort_lookahead_estimate(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=10)
        end_time = time.time()
        our_time3 = end_time - start_time

        # Measure time for Qiskit method
        start_time = time.time()
        origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
        origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        qiskit_time = end_time - start_time

        # Measure time for RustiQ method
        start_time = time.time()
        circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
        end_time = time.time()
        rustiq_time = end_time - start_time
        
        # Collect results
        result = {
            "num_paulis": len(test_paulis),
            "times": {
                "our_time": our_time,
                "combined_time": combined_time,
                "sort_time": our_time3,
                "qiskit_time": qiskit_time,
                "rustiq_time": rustiq_time
            },
            "gate_counts": {
                "our_method": opt_qc_f.count_ops().get('cx', 0),
                "sort_method": opt_qc_f3.count_ops().get('cx', 0),
                "combined_method": opt_qiskit.count_ops().get('cx', 0),
                "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                "rustiq_method": entangling_count(circuit)
            },
            "test_paulis_file": f'benchmarks/test_' + filename
        }
        print(result)
        results.append(result)

        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/test_' + filename, 'w') as paulis_file:
            json.dump(test_paulis, paulis_file, indent=4)
        
# Write results to a JSON file
with open('benchmarks/maxcut_results.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)
    


max_cut_benchmark_n10_e45_l20.json
{'num_paulis': 1100, 'times': {'our_time': 35.351526975631714, 'combined_time': 34.32901191711426, 'sort_time': 26.01653504371643, 'qiskit_time': 1.8657548427581787, 'rustiq_time': 0.5532290935516357}, 'gate_counts': {'our_method': 1080, 'sort_method': 988, 'combined_method': 1080, 'qiskit_method': 1798, 'rustiq_method': 1119}, 'test_paulis_file': 'benchmarks/test_max_cut_benchmark_n10_e45_l20.json'}


In [14]:
#newlookahead
#Compare Hamiltonian simulation:
import os, json

# Specify the folder containing the JSON files
folder_path = "benchmarks/max_cut_paulis"

results = []
# Iterate over all files in the folder
for filename in ["max_cut_benchmark_n10_e45_l20.json"]:#os.listdir(folder_path): #["max_cut_benchmark_n10_e20.json"]
    # Check if the file is a JSON file
    if filename.endswith(".json"):
        # Print the filename
        print(filename)
        with open(folder_path + '/' + filename, "r") as file:
            paulis = json.load(file)
        #expand them
        # flattened_paulis = [item for sublist in paulis for item in sublist]
        # Function to check if a string contains only 'I' characters
        def is_all_identity(pauli):
            return all(char == 'I' for char in pauli)

        # Filter the list to remove all identity Paulis
        length = len(paulis)
        test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)] 
        test_params = [0.01 * i for i in range(len(test_paulis))]

        # Measure time for our method
        start_time = time.time()
        opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=20)
        end_time = time.time()
        our_time = end_time - start_time

        start_time = time.time()
        opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=20)
        opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        combined_time = end_time - start_time
        
        # Measure time for our method
        start_time = time.time()
        opt_qc_f3, append_clifford_f3, sorted_entanglers_f3 = fc_tree_sort_lookahead_estimate(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=20)
        end_time = time.time()
        our_time3 = end_time - start_time

        # Measure time for Qiskit method
        start_time = time.time()
        origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
        origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        qiskit_time = end_time - start_time

        # Measure time for RustiQ method
        start_time = time.time()
        circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
        end_time = time.time()
        rustiq_time = end_time - start_time
        
        # Collect results
        result = {
            "num_paulis": len(test_paulis),
            "times": {
                "our_time": our_time,
                "combined_time": combined_time,
                "sort_time": our_time3,
                "qiskit_time": qiskit_time,
                "rustiq_time": rustiq_time
            },
            "gate_counts": {
                "our_method": opt_qc_f.count_ops().get('cx', 0),
                "sort_method": opt_qc_f3.count_ops().get('cx', 0),
                "combined_method": opt_qiskit.count_ops().get('cx', 0),
                "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                "rustiq_method": entangling_count(circuit)
            },
            "test_paulis_file": f'benchmarks/test_' + filename
        }
        print(result)
        results.append(result)

        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/test_' + filename, 'w') as paulis_file:
            json.dump(test_paulis, paulis_file, indent=4)
        
# Write results to a JSON file
with open('benchmarks/maxcut_results.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)
    


max_cut_benchmark_n10_e45_l20.json
{'num_paulis': 1100, 'times': {'our_time': 48.19478797912598, 'combined_time': 44.24647498130798, 'sort_time': 35.867356061935425, 'qiskit_time': 2.1197080612182617, 'rustiq_time': 0.6992740631103516}, 'gate_counts': {'our_method': 1086, 'sort_method': 1001, 'combined_method': 1085, 'qiskit_method': 1798, 'rustiq_method': 1119}, 'test_paulis_file': 'benchmarks/test_max_cut_benchmark_n10_e45_l20.json'}


In [13]:
#newlookahead
#Compare Hamiltonian simulation:
import os, json

# Specify the folder containing the JSON files
folder_path = "benchmarks/max_cut_paulis"

results = []
# Iterate over all files in the folder
for filename in ["max_cut_benchmark_n20_e95_l40.json"]:#os.listdir(folder_path): #["max_cut_benchmark_n10_e20.json"]
    # Check if the file is a JSON file
    if filename.endswith(".json"):
        # Print the filename
        print(filename)
        with open(folder_path + '/' + filename, "r") as file:
            paulis = json.load(file)
        #expand them
        # flattened_paulis = [item for sublist in paulis for item in sublist]
        # Function to check if a string contains only 'I' characters
        def is_all_identity(pauli):
            return all(char == 'I' for char in pauli)

        # Filter the list to remove all identity Paulis
        length = len(paulis)
        test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)] 
        test_params = [0.01 * i for i in range(len(test_paulis))]

        # Measure time for our method
        start_time = time.time()
        opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=10)
        end_time = time.time()
        our_time = end_time - start_time

        start_time = time.time()
        opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=10)
        opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        combined_time = end_time - start_time
        
        # Measure time for our method
        start_time = time.time()
        opt_qc_f3, append_clifford_f3, sorted_entanglers_f3 = fc_tree_sort_lookahead_estimate(entanglers=test_paulis, params=test_params, barrier=False, lookahead_size=10)
        end_time = time.time()
        our_time3 = end_time - start_time

        # Measure time for Qiskit method
        start_time = time.time()
        origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
        origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
        end_time = time.time()
        qiskit_time = end_time - start_time

        # Measure time for RustiQ method
        start_time = time.time()
        circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
        end_time = time.time()
        rustiq_time = end_time - start_time
        
        # Collect results
        result = {
            "num_paulis": len(test_paulis),
            "times": {
                "our_time": our_time,
                "combined_time": combined_time,
                "sort_time": our_time3,
                "qiskit_time": qiskit_time,
                "rustiq_time": rustiq_time
            },
            "gate_counts": {
                "our_method": opt_qc_f.count_ops().get('cx', 0),
                "sort_method": opt_qc_f3.count_ops().get('cx', 0),
                "combined_method": opt_qiskit.count_ops().get('cx', 0),
                "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                "rustiq_method": entangling_count(circuit)
            },
            "test_paulis_file": f'benchmarks/test_' + filename
        }
        print(result)
        results.append(result)

        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/test_' + filename, 'w') as paulis_file:
            json.dump(test_paulis, paulis_file, indent=4)
        
# Write results to a JSON file
with open('benchmarks/maxcut_results.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)
    


max_cut_benchmark_n20_e95_l40.json
{'num_paulis': 4600, 'times': {'our_time': 451.8365430831909, 'combined_time': 461.43796396255493, 'sort_time': 449.88276195526123, 'qiskit_time': 8.13303804397583, 'rustiq_time': 20.515342950820923}, 'gate_counts': {'our_method': 6717, 'sort_method': 6359, 'combined_method': 6716, 'qiskit_method': 7438, 'rustiq_method': 6385}, 'test_paulis_file': 'benchmarks/test_max_cut_benchmark_n20_e95_l40.json'}


In [6]:
opt_qc_f.count_ops()

OrderedDict([('rz', 2000), ('h', 39)])

In [11]:
sorted_entanglers_f

[[['IIIIZIIIIZ', 0.0],
  ['IZZIIIIIII', 0.03],
  ['ZIZIIIIIII', 0.05],
  ['ZIIIIIIIZI', 0.11],
  ['ZIIZIIIIII', 0.19],
  ['IIZZIIIIII', 0.14],
  ['IIIZIIIIII', 0.18],
  ['ZIZIIZIIII', 0.02],
  ['IIZIIIZIII', 0.06],
  ['IZIIIIZIII', 0.09],
  ['ZIIIZIIIIZ', 0.04],
  ['ZIIIIIIIZI', 0.12],
  ['IIZIIIIZII', 0.13],
  ['IIIZIIIZII', 0.01],
  ['IIIZIIIIII', 0.16],
  ['IZIIIIZZII', 0.08],
  ['IIZIZIIIII', 0.17],
  ['IIIIZZIIII', 0.1],
  ['IIZIIZIIIZ', 0.07],
  ['IZZZIIIZII', 0.15]],
 [['IIIIIIIIXI', 0.2],
  ['IIIIIIXIII', 0.21],
  ['IXIXIIIIII', 0.23],
  ['IIIIXIIIII', 0.24],
  ['XIIIIIIIZI', 0.28],
  ['IZXZIIIIII', 0.22],
  ['IIIIZXIIII', 0.25],
  ['IIZIIIZXII', 0.26],
  ['IIIZIIIZII', 0.27],
  ['ZZZIZZIIZX', 0.29]],
 [['XIIIIIIIII', 0.42],
  ['IXIIIIIIII', 0.49],
  ['IIIXIIIIII', 0.31],
  ['IIIIIIXIII', 0.39],
  ['IIIIIXIIII', 0.4],
  ['IIIZIIIIII', 0.46],
  ['IIIIXIIIII', 0.47000000000000003],
  ['ZIXIZIIIII', 0.3],
  ['ZIIIIZIIII', 0.37],
  ['ZIIIIIIIXI', 0.34],
  ['ZZIIIIIIII', 0.45],
  ['

In [12]:
sorted_entanglers_f3

[[['IIIIZIIIIZ', 0.0],
  ['IIIZIIIZII', 0.01],
  ['IIIZIIIIII', 0.16],
  ['IIZIIZIIII', 0.02],
  ['IZZIIIIIII', 0.03],
  ['ZIIIZIIIIZ', 0.04],
  ['ZZZIIIIIII', 0.05],
  ['IIZIIIZIII', 0.06],
  ['ZIIIZZIIII', 0.07],
  ['IIIIIIZZII', 0.08],
  ['IZIIIIZIII', 0.09],
  ['ZIIIIIIIIZ', 0.1],
  ['ZZZIZIZIZI', 0.11],
  ['IZIIIZIIII', 0.12],
  ['IIZIIIIZII', 0.13],
  ['IIZZIIIIII', 0.14],
  ['IIZIIIIIII', 0.18],
  ['ZZIZIZIZII', 0.15],
  ['IIIIIZIIIZ', 0.17],
  ['ZZIIZZIIII', 0.19]],
 [['IIIIXIIIII', 0.2],
  ['IIIIIIXIII', 0.21],
  ['IIXIIXZIXX', 0.22],
  ['XXZXZIIIII', 0.23],
  ['IIIIIIIIIZ', 0.24],
  ['IIZZIZIIZZ', 0.25],
  ['ZIZIIIZXII', 0.26],
  ['IIIIIIIZII', 0.27],
  ['IZIZZZIIII', 0.28],
  ['ZZZIIIIZII', 0.29]],
 [['IIXIIIXIIX', 0.3],
  ['IIIIIIIXII', 0.31],
  ['IIIIIIIZII', 0.46],
  ['IXZXIXIZXZ', 0.32],
  ['IIIIIIZIZI', 0.33],
  ['IZIZIIZIII', 0.34],
  ['IZZIIIIIII', 0.35000000000000003],
  ['XZIIIIIIII', 0.36],
  ['IIZIIIZIII', 0.37],
  ['ZIZIIIIZZI', 0.38],
  ['ZZIZIIZIII', 0.39],
  [

In [11]:
filename

'max_cut_benchmark_n10_e20,json'

In [7]:
test_paulis

['ZZZZZZZZZZZZZZZ',
 'IIZIIIIIIIIIZII',
 'IIIIZIIIZIIIIII',
 'IIIIIIIZIIZIIII',
 'IIIIIIIIIIIZZII',
 'IZIIZIIIIIIIIII',
 'IIIIIIIIZIZIIII',
 'IIIZIIIIIIIIIIZ',
 'IIIIIIIIIIZIIZI',
 'IIIIIZIIIIIIZII',
 'IIIZIIIIIIIZIII',
 'IIIIIIIIIIIZIIZ',
 'IIIIIIIZIIIIIZI',
 'ZIZIIIIIIIIIIII',
 'IIIZIIZIIIIIIII',
 'IIZIIZIIIIIIIII',
 'IIIZIIIIIZIIIII',
 'IIZIIIIIIIIIIIZ',
 'IIIIIIIIIIZIIIZ',
 'ZIIIIIIIIIZIIII',
 'IIIIIZIZIIIIIII',
 'IIIIIIIIZIIZIII',
 'IIIIIZIIZIIIIII',
 'ZIIIIZIIIIIIIII',
 'IIIIIZZIIIIIIII',
 'IIIIIIIZIIIIIIZ',
 'IIIZIIIIIIIIZII',
 'IIIIZZIIIIIIIII',
 'IIIIIZIIIIIIIIZ',
 'IZIIIIIZIIIIIII',
 'IIIIIIZIIIIIIZI',
 'ZIIIIIZIIIIIIII',
 'IIIIIIZZIIIIIII',
 'ZZIIIIIIIIIIIII',
 'IIZIIZIIIIIIIII',
 'IIIIIIIIZZIIIII',
 'IZIIIIIIIIIIZII',
 'IIIZZIIIIIIIIII',
 'IIIIIIZIZIIIIII',
 'IIZZIIIIIIIIIII',
 'IIIIIIIIIIIZIZI',
 'ZIIZIIIIIIIIIII',
 'IZIIIIIIZIIIIII',
 'IIIIIIIIIZIIIZI',
 'IZIIIIIIIIZIIII',
 'IIIIIIIIIIIZZII',
 'IIIIZZIIIIIIIII',
 'IIZIZIIIIIIIIII',
 'ZIIZIIIIIIIIIII',
 'ZIIIIIIZIIIIIII',


In [ ]:
UCC1020, PCOAST runtime: very long